In [1]:
#IMPORT LIBRARIES

#Scipy Libraries
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
import matplotlib.pyplot as plt

#Seaborn library
import seaborn as sns


Dataset downloaded from here: https://datadashboard.fda.gov/ora/cd/inspections.htm

Specifically the inspection citations dataset

In [128]:
insp = pd.read_csv('FDA_Inspections.csv', parse_dates=['Inspection End Date'], index_col= ['Inspection End Date'])#Read in the Data
insp = pd.DataFrame(insp)#Create a dataframe

#drop the Inspection ID and FEI number columns as they are mixed data types and not relevant to the analysis
insp=insp.drop(['Inspection ID', 'FEI Number'], axis=1)

print(insp.shape)
print(insp.columns)

(220595, 6)
Index(['Legal Name', 'Program Area', 'Act/CFR Number', 'Short Description',
       'Long Description', 'Firm Profile'],
      dtype='object')


In [129]:
insp.head(5)

,Legal Name,Program Area,Act/CFR Number,Short Description,Long Description,Firm Profile
Inspection End Date,,,,,,
2021-06-16,ARMS GLOBAL INC,Foods,21 CFR 1.502(a),Develop FSVP,You did not develop an FSVP.,https://datadashboard.fda.gov/ora/firmprofile....
2021-06-15,Mesada International Trading,Foods,21 CFR 1.502(a),Develop FSVP,You did not develop an FSVP.,https://datadashboard.fda.gov/ora/firmprofile....
2021-06-10,ImaCor Inc.,Devices,21 CFR 803.17,Lack of Written MDR Procedures,Written MDR procedures have not been maintained.,https://datadashboard.fda.gov/ora/firmprofile....
2021-06-10,ImaCor Inc.,Devices,21 CFR 820.40(a),"Document review, approval documentation",The documentation of approval of documents doe...,https://datadashboard.fda.gov/ora/firmprofile....
2021-06-10,ImaCor Inc.,Devices,21 CFR 820.198(a),Complaints,Complaint files are not adequately maintained.,https://datadashboard.fda.gov/ora/firmprofile....


### SPLIT THE DATASET INTO EACH INSPECTION CATEGORY

LIST EACH INSPECTION CATEGORY

In [130]:
#Dropping duplicates will show each unique area of inspection
unique_cats = insp.drop_duplicates(subset = ["Program Area"])
unique_cats

,Legal Name,Program Area,Act/CFR Number,Short Description,Long Description,Firm Profile
Inspection End Date,,,,,,
2021-06-16,ARMS GLOBAL INC,Foods,21 CFR 1.502(a),Develop FSVP,You did not develop an FSVP.,https://datadashboard.fda.gov/ora/firmprofile....
2021-06-10,ImaCor Inc.,Devices,21 CFR 803.17,Lack of Written MDR Procedures,Written MDR procedures have not been maintained.,https://datadashboard.fda.gov/ora/firmprofile....
2021-05-28,"Bravo Packing, Inc.",Veterinary Medicine,21 CFR 507.14,Personnel,You did not take reasonable measures and preca...,https://datadashboard.fda.gov/ora/firmprofile....
2021-05-21,"Fresenius Medical Care, North America",Drugs,21 CFR 211.192,Written record of investigation incomplete,Written records of investigations into unexpla...,https://datadashboard.fda.gov/ora/firmprofile....
2021-05-18,"U.S. Dept. of Veterans Affairs, Northport VA M...",Biologics,21 CFR 600.12(a),Maintenance - concurrence,Records are not made concurrently with the per...,https://datadashboard.fda.gov/ora/firmprofile....
2021-05-03,"Chafic Karam, M.D.",Bioresearch Monitoring,21 CFR 312.60,"FD-1572, protocol compliance",An investigation was not conducted in accordan...,https://datadashboard.fda.gov/ora/firmprofile....
2021-03-16,Southwest Fertility Center,Human Tissue for Transplantation,21 CFR 1271.85(a),Infection with communicable disease agents,Donors were not tested for evidence of infecti...,https://datadashboard.fda.gov/ora/firmprofile....
2021-02-25,Modesto Food Distributors dba Onorato & Compan...,Part 11 Compliance,NaN,Procedural requirements,Deviation from the procedural requirements of ...,https://datadashboard.fda.gov/ora/firmprofile....
2020-03-05,"Mevion Medical Systems, Inc.",Radiologic Health,21 CFR 1002.13,Failure to submit,You did not submit an annual report for produc...,https://datadashboard.fda.gov/ora/firmprofile....


In [131]:
Foods = insp[insp["Program Area"] == 'Foods']
Foods.head(5)

,Legal Name,Program Area,Act/CFR Number,Short Description,Long Description,Firm Profile
Inspection End Date,,,,,,
2021-06-16,ARMS GLOBAL INC,Foods,21 CFR 1.502(a),Develop FSVP,You did not develop an FSVP.,https://datadashboard.fda.gov/ora/firmprofile....
2021-06-15,Mesada International Trading,Foods,21 CFR 1.502(a),Develop FSVP,You did not develop an FSVP.,https://datadashboard.fda.gov/ora/firmprofile....
2021-06-09,Fresh Goods Trading Corporation,Foods,21 CFR 1.502(a),Develop FSVP,You did not develop an FSVP.,https://datadashboard.fda.gov/ora/firmprofile....
2021-06-09,Mutti USA Inc.,Foods,21 CFR 1.502(a),Develop FSVP,You did not develop an FSVP.,https://datadashboard.fda.gov/ora/firmprofile....
2021-06-07,Oax Imports Corp,Foods,21 CFR 1.502(a),Develop FSVP,You did not develop an FSVP.,https://datadashboard.fda.gov/ora/firmprofile....


In [132]:
Devices = insp[insp["Program Area"] == 'Devices']
Devices.head(5)

,Legal Name,Program Area,Act/CFR Number,Short Description,Long Description,Firm Profile
Inspection End Date,,,,,,
2021-06-10,ImaCor Inc.,Devices,21 CFR 803.17,Lack of Written MDR Procedures,Written MDR procedures have not been maintained.,https://datadashboard.fda.gov/ora/firmprofile....
2021-06-10,ImaCor Inc.,Devices,21 CFR 820.40(a),"Document review, approval documentation",The documentation of approval of documents doe...,https://datadashboard.fda.gov/ora/firmprofile....
2021-06-10,ImaCor Inc.,Devices,21 CFR 820.198(a),Complaints,Complaint files are not adequately maintained.,https://datadashboard.fda.gov/ora/firmprofile....
2021-06-04,Alliance Precision Plastics Corporation,Devices,21 CFR 820.70(i),Software validation for automated processes,Software used as part of production has not be...,https://datadashboard.fda.gov/ora/firmprofile....
2021-06-04,Alliance Precision Plastics Corporation,Devices,21 CFR 820.80(c),Lack of or inadequate In-process acceptance pr...,Procedures for the acceptance of in-process pr...,https://datadashboard.fda.gov/ora/firmprofile....


In [133]:
Vet = insp[insp["Program Area"] == 'Veterinary Medicine']
Vet.head(5)

,Legal Name,Program Area,Act/CFR Number,Short Description,Long Description,Firm Profile
Inspection End Date,,,,,,
2021-05-28,"Bravo Packing, Inc.",Veterinary Medicine,21 CFR 507.14,Personnel,You did not take reasonable measures and preca...,https://datadashboard.fda.gov/ora/firmprofile....
2021-05-28,"Bravo Packing, Inc.",Veterinary Medicine,21 CFR 507.19(a),Plant Maintenance,You did not maintain your plant in a clean man...,https://datadashboard.fda.gov/ora/firmprofile....
2021-05-28,"Bravo Packing, Inc.",Veterinary Medicine,21 CFR 507.19(b),Sanitation,"You did not clean, maintain and store utensils...",https://datadashboard.fda.gov/ora/firmprofile....
2021-05-28,"Bravo Packing, Inc.",Veterinary Medicine,21 CFR 507.19(e),Pest Control,You did not take effective measures to exclude...,https://datadashboard.fda.gov/ora/firmprofile....
2021-05-20,"Lonsdale Feed Mill, Inc.",Veterinary Medicine,21 CFR 507.19(a),Plant Maintenance,You did not maintain your plant in a clean man...,https://datadashboard.fda.gov/ora/firmprofile....


In [134]:
Drugs = insp[insp["Program Area"] == 'Drugs']
Drugs.head(5)

,Legal Name,Program Area,Act/CFR Number,Short Description,Long Description,Firm Profile
Inspection End Date,,,,,,
2021-05-21,"Fresenius Medical Care, North America",Drugs,21 CFR 211.192,Written record of investigation incomplete,Written records of investigations into unexpla...,https://datadashboard.fda.gov/ora/firmprofile....
2021-05-06,"BRENNTAG PACIFIC, INC.",Drugs,21 CFR 211.22(d),"Procedures not in writing, fully followed",The responsibilities and procedures applicable...,https://datadashboard.fda.gov/ora/firmprofile....
2021-05-06,"BRENNTAG PACIFIC, INC.",Drugs,21 CFR 211.25(a),"Training , Education , Experience overall","Employees engaged in the manufacture, processi...",https://datadashboard.fda.gov/ora/firmprofile....
2021-05-06,"BRENNTAG PACIFIC, INC.",Drugs,21 CFR 211.100(a),Absence of Written Procedures,There are no written procedures for production...,https://datadashboard.fda.gov/ora/firmprofile....
2021-05-04,CMIC CMO USA Corporation,Drugs,21 CFR 211.67(c),Cleaning/maintenance records not kept,Records are not kept for the maintenance and i...,https://datadashboard.fda.gov/ora/firmprofile....


In [135]:
Bio = insp[insp["Program Area"] == 'Biologics']
Bio.head(5)

,Legal Name,Program Area,Act/CFR Number,Short Description,Long Description,Firm Profile
Inspection End Date,,,,,,
2021-05-18,"U.S. Dept. of Veterans Affairs, Northport VA M...",Biologics,21 CFR 600.12(a),Maintenance - concurrence,Records are not made concurrently with the per...,https://datadashboard.fda.gov/ora/firmprofile....
2021-05-18,"U.S. Dept. of Veterans Affairs, Northport VA M...",Biologics,21 CFR 606.160(b),Required records,Failure to maintain processing and general rec...,https://datadashboard.fda.gov/ora/firmprofile....
2021-02-24,New York Blood Center Inc,Biologics,21 CFR 606.60(b),Equipment calibration frequency,Equipment used in the collection of blood and ...,https://datadashboard.fda.gov/ora/firmprofile....
2021-02-24,New York Blood Center Inc,Biologics,21 CFR 606.100(b),"Establish, maintain and follow manufacturing SOPs",Written standard operating procedures includin...,https://datadashboard.fda.gov/ora/firmprofile....
2021-02-24,New York Blood Center Inc,Biologics,21 CFR 606.100(b),Written SOPs available for use by personnel,Failure to make available written procedures f...,https://datadashboard.fda.gov/ora/firmprofile....


In [136]:
Bio_Research = insp[insp["Program Area"] == 'Bioresearch Monitoring']
Bio_Research.head(5)

,Legal Name,Program Area,Act/CFR Number,Short Description,Long Description,Firm Profile
Inspection End Date,,,,,,
2021-05-03,"Chafic Karam, M.D.",Bioresearch Monitoring,21 CFR 312.60,"FD-1572, protocol compliance",An investigation was not conducted in accordan...,https://datadashboard.fda.gov/ora/firmprofile....
2021-05-03,"Chafic Karam, M.D.",Bioresearch Monitoring,21 CFR 312.62(b),Case history records- inadequate or inadequate,Failure to prepare or maintain accurate case h...,https://datadashboard.fda.gov/ora/firmprofile....
2021-04-23,"Robin Kroll, M.D.",Bioresearch Monitoring,21 CFR 312.60,"FD-1572, protocol compliance",An investigation was not conducted in accordan...,https://datadashboard.fda.gov/ora/firmprofile....
2021-04-23,"Robin Kroll, M.D.",Bioresearch Monitoring,21 CFR 312.62(b),Case history records- inadequate or inadequate,Failure to prepare or maintain accurate case h...,https://datadashboard.fda.gov/ora/firmprofile....
2021-04-14,"May T. Cho, M.D.",Bioresearch Monitoring,21 CFR 312.60,"FD-1572, protocol compliance",An investigation was not conducted in accordan...,https://datadashboard.fda.gov/ora/firmprofile....


In [137]:
Human_tiss = insp[insp["Program Area"] == 'Human Tissue for Transplantation']
Human_tiss.head(5)

,Legal Name,Program Area,Act/CFR Number,Short Description,Long Description,Firm Profile
Inspection End Date,,,,,,
2021-03-16,Southwest Fertility Center,Human Tissue for Transplantation,21 CFR 1271.85(a),Infection with communicable disease agents,Donors were not tested for evidence of infecti...,https://datadashboard.fda.gov/ora/firmprofile....
2021-03-04,Caribbean Dental Products Corp.,Human Tissue for Transplantation,21 CFR 1271.260(e),"Storage temperatures recorded, maintained",Storage temperatures of HCT/Ps were not record...,https://datadashboard.fda.gov/ora/firmprofile....
2020-10-16,Repro Lab,Human Tissue for Transplantation,21 CFR 1271.47(a),"Testing, screening, donor eligibility procedures",You did not establish and maintain procedures ...,https://datadashboard.fda.gov/ora/firmprofile....
2020-10-16,Repro Lab,Human Tissue for Transplantation,21 CFR 1271.50(a),Determination based on screening and testing,HCT/P donors were not determined to be eligibl...,https://datadashboard.fda.gov/ora/firmprofile....
2020-10-16,Repro Lab,Human Tissue for Transplantation,21 CFR 1271.55(b)(4),Reasons for ineligibility on summary,The summary of records for HCT/Ps from donors ...,https://datadashboard.fda.gov/ora/firmprofile....


In [138]:
Part_11 = insp[insp["Program Area"] == 'Part 11 Compliance']
Part_11.head(5)

,Legal Name,Program Area,Act/CFR Number,Short Description,Long Description,Firm Profile
Inspection End Date,,,,,,
2021-02-25,Modesto Food Distributors dba Onorato & Compan...,Part 11 Compliance,NaN,Procedural requirements,Deviation from the procedural requirements of ...,https://datadashboard.fda.gov/ora/firmprofile....
2021-01-21,Wa Heng Dou Fu and Soy Sauce Corporation,Part 11 Compliance,NaN,Procedural requirements,Deviation from the procedural requirements of ...,https://datadashboard.fda.gov/ora/firmprofile....
2021-01-20,Han Yang Corp.,Part 11 Compliance,NaN,Procedural requirements,Deviation from the procedural requirements of ...,https://datadashboard.fda.gov/ora/firmprofile....
2021-01-15,"D & M Smoked Fish, Inc.",Part 11 Compliance,NaN,Procedural requirements,Deviation from the procedural requirements of ...,https://datadashboard.fda.gov/ora/firmprofile....
2020-08-20,"Monica E. Kleinman, M.D.",Part 11 Compliance,21 CFR 11,Electronic records,"Electronic records are used, but they do not m...",https://datadashboard.fda.gov/ora/firmprofile....


In [139]:
Rad_Health = insp[insp["Program Area"] == 'Radiologic Health']
Rad_Health.head(5)

,Legal Name,Program Area,Act/CFR Number,Short Description,Long Description,Firm Profile
Inspection End Date,,,,,,
2020-03-05,"Mevion Medical Systems, Inc.",Radiologic Health,21 CFR 1002.13,Failure to submit,You did not submit an annual report for produc...,https://datadashboard.fda.gov/ora/firmprofile....
2020-03-05,"Mevion Medical Systems, Inc.",Radiologic Health,21 CFR 1002.30(a)(2),Results of tests,You have not maintained records of the results...,https://datadashboard.fda.gov/ora/firmprofile....
2020-02-24,Technological Medical Advancements LLC,Radiologic Health,21 CFR 1002.13,Failure to submit,You did not submit an annual report for produc...,https://datadashboard.fda.gov/ora/firmprofile....
2020-02-24,Technological Medical Advancements LLC,Radiologic Health,21 CFR 1010.2(b),Certification other than by tag or label,Certification of a product is not in conforman...,https://datadashboard.fda.gov/ora/firmprofile....
2020-02-24,Technological Medical Advancements LLC,Radiologic Health,21 CFR 1040.10(f)(5)(ii),Emission indicator - Class IIIb & IV,An emission indicator which signals sufficient...,https://datadashboard.fda.gov/ora/firmprofile....


In [140]:
Parts_1240_1250 = insp[insp["Program Area"] == 'Parts 1240 and 1250']
Parts_1240_1250.head(5)

,Legal Name,Program Area,Act/CFR Number,Short Description,Long Description,Firm Profile
Inspection End Date,,,,,,
2020-02-11,"Borenstein Caterers, Inc.",Parts 1240 and 1250,21 CFR 1250.30(d),"Plumbing design, installation, maintenance",Plumbing is not maintained so as to prevent co...,https://datadashboard.fda.gov/ora/firmprofile....
2020-02-11,"Borenstein Caterers, Inc.",Parts 1240 and 1250,21 CFR 1250.33(a),Maintained in good repair,Not all working surfaces used in connection wi...,https://datadashboard.fda.gov/ora/firmprofile....
2019-12-17,New Orleans Aviation Board,Parts 1240 and 1250,21 CFR 1250.67,Prevention of contamination,Failure to maintain servicing area piping syst...,https://datadashboard.fda.gov/ora/firmprofile....
2019-12-12,"Port Of Grays Harbor Terminal 1, 2 and 4",Parts 1240 and 1250,21 CFR 1250.67,Prevention of contamination,Failure to construct and maintain servicing ar...,https://datadashboard.fda.gov/ora/firmprofile....
2019-12-12,Port of Grays Harbor Terminal 3,Parts 1240 and 1250,21 CFR 1250.67,Prevention of contamination,Failure to maintain servicing area hydrants in...,https://datadashboard.fda.gov/ora/firmprofile....


#### FOCUS ON BIOLOGICS DATA SUB-SET

In [141]:
print(Bio.shape, 'Rows, Columns')

(4153, 6) Rows, Columns


Above summary indicates 4153 entries for Biologics to date from 2008 to 2021

In [142]:
bio_unique_companies = Bio.drop_duplicates(subset = ["Legal Name"])
bio_unique_companies

,Legal Name,Program Area,Act/CFR Number,Short Description,Long Description,Firm Profile
Inspection End Date,,,,,,
2021-05-18,"U.S. Dept. of Veterans Affairs, Northport VA M...",Biologics,21 CFR 600.12(a),Maintenance - concurrence,Records are not made concurrently with the per...,https://datadashboard.fda.gov/ora/firmprofile....
2021-02-24,New York Blood Center Inc,Biologics,21 CFR 606.60(b),Equipment calibration frequency,Equipment used in the collection of blood and ...,https://datadashboard.fda.gov/ora/firmprofile....
2020-11-09,New York Blood Center Enterprises,Biologics,21 CFR 606.100(b),"Establish, maintain and follow manufacturing SOPs",Written standard operating procedures includin...,https://datadashboard.fda.gov/ora/firmprofile....
2020-10-21,"BPC Plasma, Inc.",Biologics,21 CFR 630.30(a),Donation suitability,Failure to determine the suitability of a dona...,https://datadashboard.fda.gov/ora/firmprofile....
2020-03-12,"Hemarus, LLC",Biologics,21 CFR 606.60(a),"Equipment observed, standardized, calibrated",Equipment used in the collection of blood and ...,https://datadashboard.fda.gov/ora/firmprofile....
...,...,...,...,...,...,...
2008-12-08,Depaul Community Health Center,Biologics,21 CFR 606.170(a),Adverse Reaction - Investigations,A thorough investigation of each reported adve...,https://datadashboard.fda.gov/ora/firmprofile....
2008-12-05,"Dorado Health, Inc.dba Manati Medical Center",Biologics,21 CFR 606.100(b),"Establish, maintain and follow manufacturing SOPs",Written standard operating procedures includin...,https://datadashboard.fda.gov/ora/firmprofile....
2008-12-04,"Department of Air Force, 59th Medical Wing/SGVLB",Biologics,21 CFR 606.20(b),Qualifications of responsible personnel,The personnel responsible for the processing a...,https://datadashboard.fda.gov/ora/firmprofile....


In [143]:
#Convert Biologics Subset into a pandas dataframe
Bio=pd.DataFrame(Bio)
Bio

,Legal Name,Program Area,Act/CFR Number,Short Description,Long Description,Firm Profile
Inspection End Date,,,,,,
2021-05-18,"U.S. Dept. of Veterans Affairs, Northport VA M...",Biologics,21 CFR 600.12(a),Maintenance - concurrence,Records are not made concurrently with the per...,https://datadashboard.fda.gov/ora/firmprofile....
2021-05-18,"U.S. Dept. of Veterans Affairs, Northport VA M...",Biologics,21 CFR 606.160(b),Required records,Failure to maintain processing and general rec...,https://datadashboard.fda.gov/ora/firmprofile....
2021-02-24,New York Blood Center Inc,Biologics,21 CFR 606.60(b),Equipment calibration frequency,Equipment used in the collection of blood and ...,https://datadashboard.fda.gov/ora/firmprofile....
2021-02-24,New York Blood Center Inc,Biologics,21 CFR 606.100(b),"Establish, maintain and follow manufacturing SOPs",Written standard operating procedures includin...,https://datadashboard.fda.gov/ora/firmprofile....
2021-02-24,New York Blood Center Inc,Biologics,21 CFR 606.100(b),Written SOPs available for use by personnel,Failure to make available written procedures f...,https://datadashboard.fda.gov/ora/firmprofile....
...,...,...,...,...,...,...
2008-10-17,American Red Cross Blood Services Heart of Ame...,Biologics,21 CFR 606.100(c),Thorough investigations,Failure to perform a thorough investigation of...,https://datadashboard.fda.gov/ora/firmprofile....
2008-10-17,American Red Cross Blood Services Heart of Ame...,Biologics,21 CFR 606.170(a),Adverse Reaction - Investigations,A thorough investigation of each reported adve...,https://datadashboard.fda.gov/ora/firmprofile....
2008-10-10,"Lonza Walkersville, Inc.",Biologics,21 CFR 600.14(a)(1),Who must report - manufacturer,Failure to submit biological deviation reports.,https://datadashboard.fda.gov/ora/firmprofile....


In [150]:
Bio['2020']

<ipython-input-150-43763714b8f3>:1: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  Bio['2020']


,Legal Name,Program Area,Act/CFR Number,Short Description,Long Description,Firm Profile
Inspection End Date,,,,,,
2020-11-09,New York Blood Center Enterprises,Biologics,21 CFR 606.100(b),"Establish, maintain and follow manufacturing SOPs",Written standard operating procedures includin...,https://datadashboard.fda.gov/ora/firmprofile....
2020-10-21,"BPC Plasma, Inc.",Biologics,21 CFR 630.30(a),Donation suitability,Failure to determine the suitability of a dona...,https://datadashboard.fda.gov/ora/firmprofile....
2020-03-12,"Hemarus, LLC",Biologics,21 CFR 606.60(a),"Equipment observed, standardized, calibrated",Equipment used in the collection of blood and ...,https://datadashboard.fda.gov/ora/firmprofile....
2020-03-12,"Hemarus, LLC",Biologics,21 CFR 630.40(b)(3),Results of testing,Failure to notify the donor of the results of ...,https://datadashboard.fda.gov/ora/firmprofile....
2020-03-12,"Hemarus, LLC",Biologics,21 CFR 640.64(e),Prevention of contamination,The phlebotomy site is not prepared by a metho...,https://datadashboard.fda.gov/ora/firmprofile....
2020-03-04,"The Community Blood Center, Inc.",Biologics,21 CFR 606.100(c),Thorough investigations,Failure to perform a thorough investigation an...,https://datadashboard.fda.gov/ora/firmprofile....
2020-02-27,Millipore (U.K.) Ltd.,Biologics,21 CFR 600.12(a),Maintenance - tracing,Records are not maintained in a manner which a...,https://datadashboard.fda.gov/ora/firmprofile....
2020-02-27,Millipore (U.K.) Ltd.,Biologics,21 CFR 610.1,Tests prior to release,Lots of licensed products were released by the...,https://datadashboard.fda.gov/ora/firmprofile....
2020-02-27,American Red Cross Blood Svcs,Biologics,21 CFR 606.40,Clean & orderly,Failure to maintain facilities in a clean and ...,https://datadashboard.fda.gov/ora/firmprofile....
